# General Chat Agent

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List, Optional, Any
from langchain.tools import tool, Tool
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.chat_models.anthropic import ChatAnthropic
from langchain.chat_models.openai import ChatOpenAI
from langchain.automaton.typedefs import MessageLog
from langchain.automaton.chat_agent import ChatAgent
from langchain.automaton.tool_utils import generate_tool_info
from langchain.prompts import ChatPromptTemplate

In [3]:
def get_tools() -> List[Tool]:
    @tool
    def name() -> str:
        """Use to look up the user's name"""
        return "Eugene"

    @tool
    def get_weather(city: str) -> str:
        """Get weather in a specific city."""
        return "42F and sunny"

    @tool
    def add(x: int, y: int) -> int:
        """Use to add two numbers. For example; { "x": 2, "y": 10}"""
        return x + y

    @tool
    def add_3(x: int, y: int, z: int) -> int:
        """Add 3 numbers together."""
        return x + y + z

    @tool
    def divide(x: float, y: float) -> float:
        """Divide x by y"""
        return x / y

    return list(locals().values())


tools = get_tools()

In [4]:
llm = ChatAnthropic(temperature=0)

In [5]:
tool_info = generate_tool_info(tools)

Generate a prompt with an example trace

In [6]:
sys_msg = """Respond to the human as helpfully and accurately as \
possible. You have access to the following tools:
{tools_description}

Use a blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $BLOB, as shown.

<action>
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
</action>

When invoking a tool do not provide any clarifying information.

The human will forward results of tool invocations as "Observations".

When you know the answer paraphrase the information in the observations properly and respond to the user. \
If you do not know the answer use more tools.

You can only take a single action at a time."""

messages = ChatPromptTemplate.from_messages(
    [
        ("system", sys_msg),
        ("human", "what is 5+8"),
        (
            "ai",
            '<action> {{ "action": "add", "action_input": {{ "x": 5, "y": 8 }} }} </action>',
        ),
        ("human", "Observation: 13"),
        ("ai", "5 + 8 is 13"),
    ]
).format_messages(**tool_info)

In [7]:
from langchain.callbacks.manager import (
    trace_as_chain_group,
    atrace_as_chain_group,
)

In [8]:
agent = ChatAgent(llm, tools)

In [9]:
class VerboseMessageLog(MessageLog):
    def add_messages(self, *args, **kwargs):
        for arg in args:
            print(arg)
        return super().add_messages(*args, **kwargs)

In [11]:
message_log = VerboseMessageLog(messages=messages)
question = HumanMessage(
    content="Sum of first 10 numbers starting from 1? use one tool at a time"
)
message_log.add_messages([question])

agent.run(message_log)

[human: Sum of first 10 numbers starting from 1? use one tool at a time]
[ai:  <action> 
{
  "action": "add",
  "action_input": {
    "x": 1, 
    "y": 2
  }
}
</action>, FunctionCall(name='add', arguments={'x': 1, 'y': 2}), FunctionResult(name='add', result=3, error=None)]
[ai:  <action>
{
  "action": "add", 
  "action_input": {
    "x": 3,
    "y": 3
  }
}
</action>, FunctionCall(name='add', arguments={'x': 3, 'y': 3}), FunctionResult(name='add', result=6, error=None)]
[ai:  <action>
{
  "action": "add",
  "action_input": {
    "x": 6, 
    "y": 4
  }  
}
</action>, FunctionCall(name='add', arguments={'x': 6, 'y': 4}), FunctionResult(name='add', result=10, error=None)]
[ai:  <action>
{
  "action": "add",
  "action_input": {
    "x": 10,
    "y": 5
  }
}  
</action>, FunctionCall(name='add', arguments={'x': 10, 'y': 5}), FunctionResult(name='add', result=15, error=None)]
[ai:  Based on the observations so far, the sum of the first 10 numbers starting from 1 is 15. I will continue invo

# Visualize

In [36]:
import html
from langchain.automaton.chat_agent import prompt_generator

In [37]:
messages = prompt_generator(message_log)

In [39]:
# Generate HTML divs
html_divs = []
for idx, msg in enumerate(messages):
    background_color = "DodgerBlue" if idx % 2 == 0 else "Tomato"
    div_content = f"""
    <div style="border: 1px solid black; padding: 10px; background-color:{background_color}">
        <div style="display: flex;">
            <div style="font-weight: bold; margin-right: 10px;">{html.escape(msg.type)}</div>
            <div>{html.escape(msg.content)}</div>
        </div>
    </div>
"""
    html_divs.append(div_content)

# Display HTML divs in a Jupyter Notebook cell
from IPython.display import HTML, display

html_output = "\n".join(html_divs)
display(HTML(html_output))